# Pipeline

Nous suivrons le pipeline ci-dessous :

<img src="pipeline.jpg" alt="drawing" width="1000"/>

# Checking

L'objectif de cette étape est de vérifier les données (essentiellement leur cohérence, et leur qualité).

In [ ]:
# Le module pandas permet de manipuler les données
import pandas as pd

# Les Fonctions permettant le checking des données
from check_data import data_info, stats_data, boxplot_data, features_distributions

## Statistiques descriptives

In [ ]:
# Charger les données sous forme de tuple
dataset = pd.read_csv("train_set.csv")

# Afficher un apperçu des données
display(dataset.head())
data_info(dataset)
stats_data(dataset)

Nous avons trois classes 0, 1, 2. Chaque classe représente une epsèce d'Iris, les codes sont les suivant :  

* 0 = ***setosa***
* 1 = ***versicolor***
* 2 = ***virginica***

## Visualisation graphique des données

In [ ]:
boxplot_data(dataset)
features_distributions(dataset)

# Prétraitement

Le but est de nettoyer/optimiser les données, de sorte à les rendre plus qualitatives, et/ou cohérentes, pour simplifier l'apprentissage machine. 

In [ ]:
# La fonction MinMaxScaler permet de normaliser les données 
from sklearn.preprocessing import MinMaxScaler

# Fonction retirant les lignes dupliquées
from check_data import clean_duplicated

## Duplications

On retirer les lignes présentes en double, si elles existent :

In [ ]:
# Retrait des lignes dupliquées
dataset = clean_duplicated(dataset)

## Normalisation

On normalise ici les données d'entrainement :

In [ ]:
# Resplitter le dataset en deux
x_train, y_train = dataset.iloc[:, 0:4], dataset.iloc[:, 4:]

# Créer un objet MinMaxScaler
normalizer = MinMaxScaler()

# Normaliser les variables
x_train = normalizer.fit_transform(x_train)

# Retransformer x_train en dataframe
x_train = pd.DataFrame(data=x_train, columns=normalizer.get_feature_names_out())

# Afficher les résultats de la normalisation
display(x_train.head(), y_train.head())

On normalise ici les données données de test :

In [ ]:
# Charger le test_set
test_set = pd.read_csv("test_set.csv")

# Séparer les données en deux sets
x_test, y_test = test_set.iloc[:, 0:4], test_set.iloc[:, 4:]

# Normaliser le test_set
x_test = normalizer.transform(x_test)

# Retransformer x_test en dataframe
x_test = pd.DataFrame(data=x_test, columns=normalizer.get_feature_names_out())

# Afficher le test_set
display(x_test.head(), y_test.head())

# Entraînement

On cherche à déterminer quel est le meilleur algorithme à utiliser. Pour répondre à cette question, nous importons différents algorithmes candidats.  

In [ ]:
# Plusieurs algorithmes d'apprentissage que nous allons tester
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from eval_model import plot_ResTrainning

Pour les départager, on testera 1 à 1 les différents algorithmes (par utilisation d'une boucle *for*), en s'assurant de la reproductibilité des résultats :

In [ ]:
# Mettre les algorithmes dans un liste pour les itérer
learning_algo = [MultinomialNB(), LogisticRegression(random_state=42), SVC(random_state=42), DecisionTreeClassifier(random_state=42), KNeighborsClassifier()]

# Pour chaque algorithme
for algo in learning_algo :

    # Entraîner le modèle (instance de l'algorithme)
    tmp = algo
    tmp.fit(x_train, y_train.values.ravel())

    # Afficher les résultats
    plot_ResTrainning(tmp, x_train, y_train, x_test, y_test)

On trouve que le meilleur modèle, pour ces données, est le ***DecisionTreeClassifier***.

# Optimisation du modèle

On va, à présent, chercher les meilleurs hyperparamètres pour le ***DecisionTreeClassifier***.

In [ ]:
# Permet la recherche des meilleurs hyperparamètres
from sklearn.model_selection import GridSearchCV, KFold

# Pour l'affichage graphique
import matplotlib.pyplot as plt

# Pour afficher l'arbre de décision 
from sklearn.tree import plot_tree 

On définit les paramètres de la recherche : 

In [ ]:
# Instancer un arbre
model = DecisionTreeClassifier()

# Définir les paramètres à tester
params = {"max_depth" : [2, 3, 5, 10, 20, None],
         "min_samples_leaf" : [2, 5, 10, 20, 50, 100],
         "criterion" : ["gini", "entropy"],
         "random_state" : [i for i in range(0, 43, 1)]}

# Instancer l'itérateur pour la création du set de validation durant la recherche des meilleurs hyperparamètres
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

# Créer la grille de recherche
grid_search = GridSearchCV(estimator=model, param_grid=params, scoring="accuracy", cv=kfold, verbose=3)

# Effectuer la recherche + Afficher les résultats
grid_search.fit(x_train, y_train.values.ravel())
print("\nles meilleurs paramètres du modèle sont : {}".format(grid_search.best_params_))

On évalue le modèle optimisé :

In [ ]:
# Récupérer le meilleur modèle
model = grid_search.best_estimator_

plot_ResTrainning(model, x_train, y_train, x_test, y_test)

On analyse ici le fonctionnement du modèle, dans sa démarche de classification :

In [ ]:
# Affichage de l'arbre de décision
plt.figure(figsize=(14, 10))
_ = plot_tree(model, feature_names=x_test.columns, class_names=["Setosa", "Versicolor", "Virginica"], filled=True)
plt.show()

# Sauvegarde du modèle

Une fois toutes les étapes effectuer, on doit sauvegarder à la fois la chaîne de prétraitement, ainsi que le modèle entrainé, pour de futurs classifications.  

In [ ]:
# Permet de sauvegarder des objets python
import pickle as pk

In [ ]:
# Instancer le modèle avec les meilleurs hyperparamètres
model = DecisionTreeClassifier(criterion="entropy", max_depth=5, min_samples_leaf=5, random_state=0)

# Entraîner le modèle sur l'ensemble des données
model.fit(x_train, y_train)

# Sauvegarder le préprocesseur, il sera utiliser sur les futurs données
pk.dump(normalizer, open("preprocesseur.sav", 'wb'))

# Sauvegarder le modèle
pk.dump(model, open("model.sav", 'wb'))

# Pour allez plus loin

Documentation :
- [matplotlib](https://matplotlib.org/stable/index.html)
- [numpy](https://numpy.org/doc/)
- [pandas](https://pandas.pydata.org/docs/)
- [seaborn](https://seaborn.pydata.org/)
- [sklearn](https://scikit-learn.org/stable/)

Sites (liste non-exhaustive) :
- [kaggle](https://kaggle.com)
- [Machine Learning Mastery](https://machinelearningmastery.com/)
- [toward data science](https://towardsdatascience.com/)

Chaînes Youtubes (liste non-exhaustive) :
- [AIforyou - Morgan Gautherot](https://www.youtube.com/@AIforyouMorganGautherot) (FR)
- [Alexander Amini](https://www.youtube.com/@AAmini/videos) (ENG)
- [Machine learnia](https://www.youtube.com/@MachineLearnia) (FR)
- [Science4All](https://www.youtube.com/@Science4Allfrancais) (FR)
- [StatQuest with Josh Starmer](https://www.youtube.com/@statquest) (ENG)